In [ ]:
#Installing all required packages
!pip install gensim
!pip install spacy
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import modules we need
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
import nltk
import spacy
nltk.download('stopwords')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Reading news dataset from github
news_df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(news_df.head(5))

                                             content  target  \
0  From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1  From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2  From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3  From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4  From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   

            target_names  
0              rec.autos  
1  comp.sys.mac.hardware  
2  comp.sys.mac.hardware  
3          comp.graphics  
4              sci.space  


In [ ]:
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.extend(['from', 'subject', 're', 'edu', 'use'])
# Cleaning text
def textCleaning(text):
    # Removing extra white spaces from text
    text = re.sub(r'\s+',' ',text)
    # Removing special characters from text
    text = re.sub(r'\W',' ',str(text))
    # Remove Emails
    text = re.sub('\S*@\S*\s?', '',text)
    # Removing single characters from text
    text = re.sub(r'\s+[a-zA-Z]\s+',' ',text)
    # Removing numbers
    text = re.sub(r'\s+[0-9]*\s+',' ',text)
    #single quotes removing
    text = re.sub('[,\.!?]', '', text)
    # Converting entire text into lower case
    text = text.lower()
    # Word tokenisation
    tokens = text.split()
    # Removing stop words from text
    non_stopwords = [word for word in tokens if word not in stopwords]

    return non_stopwords

In [ ]:
data = news_df.content.values.tolist() # reading content column from dataset
clean_data = [textCleaning(sent) for sent in data] # cleaning text
print(clean_data[:1])

[['lerxst', 'wam', 'umd', 'thing', 'car', 'nntp', 'posting', 'host', 'rac3', 'wam', 'umd', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late', '60s', 'early', '70s', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(clean_data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[clean_data], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# Define functions for bigrams, trigrams and lemmatization
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Form Bigrams
bigram_words = make_bigrams(clean_data)

# Initialize spacy 'en' model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
lemmatised_text = lemmatization(bigram_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(lemmatised_text[:1])

[['thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wondering_anyone', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(lemmatised_text)

# Creating Corpus
texts = lemmatised_text

# Term Document Frequency matrix
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('wondering_anyone', 1),
  ('year', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keywords in the 5 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.010*"team" + 0.010*"game" + 0.008*"year" + 0.006*"government" + 0.006*"play" + 0.005*"gun" + 0.005*"win" + 0.005*"player" + 0.004*"state" + 0.004*"go"'), (1, '0.013*"say" + 0.013*"write" + 0.012*"people" + 0.010*"think" + 0.008*"know" + 0.008*"make" + 0.008*"line" + 0.007*"see" + 0.007*"article" + 0.006*"go"'), (2, '0.221*"ax" + 0.028*"t" + 0.028*"g" + 0.016*"_" + 0.015*"p" + 0.014*"r" + 0.007*"q" + 0.006*"m" + 0.006*"o" + 0.005*"o__o"'), (3, '0.014*"com" + 0.010*"line" + 0.009*"get" + 0.008*"organization" + 0.008*"write" + 0.006*"key" + 0.005*"article" + 0.005*"go" + 0.005*"nntp_poste" + 0.005*"car"'), (4, '0.017*"line" + 0.009*"organization" + 0.008*"use" + 0.008*"nntp_poste" + 0.008*"com" + 0.007*"host" + 0.007*"program" + 0.007*"drive" + 0.007*"get" + 0.006*"file"')]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatised_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.31873356748868

Coherence Score:  0.6082085251387609


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=92dd3dc88b4827d91ab08ff62d6e1210d83bf589cae9c73a226e5784d16ccde9
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=bcd1e43eeac5b6c23162a663f8572452d2b0525a4e3773a03500936e3f34fd59
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.162958 -0.047054       1        1  35.092472
3     -0.143238 -0.053213       2        1  23.749118
4     -0.115623 -0.132227       3        1  21.467860
0      0.009016  0.313414       4        1  12.085993
2      0.412802 -0.080921       5        1   7.604557, topic_info=         Term          Freq         Total Category  logprob  loglift
4912       ax  22291.000000  22291.000000  Default  30.0000  30.0000
5052        g   2987.000000   2987.000000  Default  29.0000  29.0000
5351        t   2825.000000   2825.000000  Default  28.0000  28.0000
310       say   6285.000000   6285.000000  Default  27.0000  27.0000
140    people   5621.000000   5621.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
4955        c    497.633486    600.115509   Topic5  -5.3123   2.3892
2787        m    599.578446    792.749920   Topic5  -5.1260   2.2971
10877      dn    445.714051    591.123870   Topic5  -5.4225   2.2941
5141        k    206.838699    220.218180   Topic5  -6.1903   2.5137
5230        n    223.797002    276.200843   Topic5  -6.1115   2.3660

[334 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
65379      3  0.004825    0el
65379      5  0.989198    0el
65422      3  0.006024    2ei
65422      5  0.987973    2ei
4778       5  0.998112    34u
...      ...       ...    ...
201        3  0.146458  write
36         1  0.234242   year
36         2  0.338706   year
36         4  0.426827   year
4385       5  0.997731      z

[505 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 1, 3])